In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

train_path = './processed_data/minmax_pca_25/train.csv'
test_path = './processed_data/minmax_pca_25/test.csv'
train = pd.read_csv(train_path, index_col=0)
test = pd.read_csv(test_path, index_col=0)

train_X = train.iloc[:,3:].to_numpy()
train_Y = train.iloc[:,:3].to_numpy()
test_X = test.iloc[:,3:].to_numpy()
test_Y = test.iloc[:,:3].to_numpy()

In [2]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error

param_test1 = {'n_estimators':range(20,81,10)}
gsearch1 = GridSearchCV(estimator = GradientBoostingRegressor(learning_rate=0.1, min_samples_split=50, min_samples_leaf=20, max_depth=6, max_features=20, subsample=0.8, random_state=10),
    param_grid = param_test1, scoring='neg_mean_squared_error', n_jobs=4, cv=5)
gsearch1.fit(train_X, train_Y[:, 1])
gsearch1.best_params_, gsearch1.best_score_

({'n_estimators': 60}, -195.068478811739)

In [3]:
param_test2 = {'max_depth':range(3,16,2), 'min_samples_split':range(80,201,20)}
gsearch2 = GridSearchCV(estimator = GradientBoostingRegressor(learning_rate=0.1, n_estimators=60, max_features=20, subsample=0.8, random_state=10), 
param_grid = param_test2, scoring='neg_mean_squared_error',n_jobs=4, cv=5)
gsearch2.fit(train_X, train_Y[:, 1])
gsearch2.best_params_, gsearch2.best_score_

({'max_depth': 7, 'min_samples_split': 140}, -193.21583235260235)

In [4]:
param_test3 = {'min_samples_leaf':range(30,151,10)}
gsearch3 = GridSearchCV(estimator = GradientBoostingRegressor(learning_rate=0.1, n_estimators=60, max_depth=7, min_samples_split=140, max_features=20, subsample=0.8, random_state=10), 
param_grid = param_test3, scoring='neg_mean_squared_error',n_jobs=4, cv=5)
gsearch3.fit(train_X, train_Y[:, 1])
gsearch3.best_params_, gsearch3.best_score_

({'min_samples_leaf': 80}, -190.68440387618602)

In [7]:
param_test4 = {'max_features':range(7,26,1)}
gsearch4 = GridSearchCV(estimator = GradientBoostingRegressor(learning_rate=0.1, n_estimators=60, max_depth=7, min_samples_split=140, min_samples_leaf=80, subsample=0.8, random_state=10),
param_grid = param_test4, scoring='neg_mean_squared_error',n_jobs=4, cv=5)
gsearch4.fit(train_X, train_Y[:, 1])
gsearch4.best_params_, gsearch4.best_score_

({'max_features': 20}, -190.68440387618602)

In [8]:
param_test5 = {'subsample':[0.6,0.7,0.75,0.8,0.85,0.9]}
gsearch5 = GridSearchCV(estimator = GradientBoostingRegressor(learning_rate=0.1, n_estimators=60, max_depth=7, min_samples_split=140, min_samples_leaf=80, max_features=20, random_state=10),
param_grid = param_test5, scoring='neg_mean_squared_error',n_jobs=4, cv=5)
gsearch5.fit(train_X, train_Y[:, 1])
gsearch5.best_params_, gsearch5.best_score_

({'subsample': 0.8}, -190.68440387618602)

In [13]:
param_test6 = {'n_estimators':[55,60,65], 'max_depth':[6,7,8], 'min_samples_split':[130,140,150]}
gsearch6 = GridSearchCV(estimator = GradientBoostingRegressor(learning_rate=0.1, random_state=10, min_samples_leaf=80, max_features=20, subsample=0.8),
param_grid = param_test6, scoring='neg_mean_squared_error',n_jobs=4, cv=5)
gsearch6.fit(train_X, train_Y[:, 1])
gsearch6.best_params_, gsearch6.best_score_

({'max_depth': 7, 'min_samples_split': 130, 'n_estimators': 60},
 -190.68440387618602)

In [12]:
param_test7 = {'min_samples_leaf':[75,80,85],'max_features':[19,20,21], 'subsample':[0.78,0.8,0.82]}
gsearch7 = GridSearchCV(estimator = GradientBoostingRegressor(learning_rate=0.1, random_state=10, max_depth=7, min_samples_split=130, n_estimators=60),
param_grid = param_test7, scoring='neg_mean_squared_error',n_jobs=4, cv=5)
gsearch7.fit(train_X, train_Y[:, 1])
gsearch7.best_params_, gsearch7.best_score_

({'max_features': 20, 'min_samples_leaf': 80, 'subsample': 0.8},
 -190.68440387618602)

In [19]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

for i in range(1,2):
    print("task",i)
    gbdt = GradientBoostingRegressor(learning_rate=0.05, n_estimators=120, max_depth=7, min_samples_split=130, min_samples_leaf=80, max_features=20, subsample=0.8, random_state=10)
    # gbdt = GradientBoostingRegressor()
    gbdt.fit(train_X, train_Y[:, i])
    mse = mean_squared_error(y_true=test_Y[:, i], y_pred=gbdt.predict(test_X))
    mae = mean_absolute_error(y_true=test_Y[:, i], y_pred=gbdt.predict(test_X))
    r2 = r2_score(y_true=test_Y[:, i], y_pred=gbdt.predict(test_X))
    print("mse: {:.2f}".format(mse))
    print("rmse: {:.2f}".format(np.sqrt(mse)))
    print("mae: {:.2f}".format(mae))
    print("r2: {:.2f}".format(r2))

# for i in range(3):
#     svr = SVR()
#     svr.fit(train_X, train_Y[:, i])
#     mse = mean_squared_error(y_true=test_Y[:, i], y_pred=svr.predict(test_X))
#     print(mse)

task 1
mse: 186.35
rmse: 13.65
mae: 10.69
r2: 0.37


In [124]:
from tqdm import tqdm

res = []
for dirname in tqdm(os.listdir('./processed_data')):
    train_path = f'./processed_data/{dirname}/train.csv'
    test_path = f'./processed_data/{dirname}/test.csv'
    train = pd.read_csv(train_path, index_col=0)
    test = pd.read_csv(test_path, index_col=0)

    train_X = train.iloc[:,3:].to_numpy()
    train_Y = train.iloc[:,:3].to_numpy()
    test_X = test.iloc[:,3:].to_numpy()
    test_Y = test.iloc[:,:3].to_numpy()
    
    gbdt_res = []
    svr_res = []
    for i in range(3):
        gbdt = GradientBoostingRegressor()
        gbdt.fit(train_X, train_Y[:, i])
        mse = mean_squared_error(y_true=test_Y[:, i], y_pred=gbdt.predict(test_X))
        # print(mse)
        gbdt_res.append(mse)

        # svr = SVR()
        # svr.fit(train_X, train_Y[:, i])
        # mse = mean_squared_error(y_true=test_Y[:, i], y_pred=svr.predict(test_X))
        # # print(mse)
        # svr_res.append(mse)
    res.append([dirname, gbdt_res, svr_res])

100%|██████████| 1/1 [00:10<00:00, 10.23s/it]


In [123]:
mn1=1000
mn2=1000
for dirname, gbdt, svr in res:
    mn1 = min(mn1, gbdt[0])
    # mn2 = min(mn2, svr[0])
    if(gbdt[0]<365): print(dirname)
print(mn1)
# print(mn2)
# res1 = res.copy()
print(res)

minmax_pca_25
363.68767458810464
[['minmax_pca_25', [363.68767458810464, 194.97712222017938, 15.253775675584683], []]]
